## Imports


In order for us to read the files, we need to import certain modules. Pandas and altair allow us to plot the data in the picologger file

In [1]:
import os
import sys
import pandas as pd
import altair as alt
from datetime import datetime,date

In [2]:
# List installed package versions
packList = ["pandas", "altair", "numpy", "itkdb"]
for p in packList:
    try:
        mod = __import__(p)
        print(mod.__name__ + " : " + mod.__version__)
    except ImportError:
        print(p + " not installed")

# Get the current working directory
thisDir = os.getcwd()

# List files in the parent directory
parentDir = os.path.abspath(os.path.join(thisDir, os.pardir))
files_in_parent = os.listdir(parentDir)
print("Files in parent directory:", files_in_parent)

pandas : 1.4.4
altair : 5.0.1
numpy : 1.21.5
itkdb not installed
Files in parent directory: ['.git', '.gitignore', '.ipynb_checkpoints', 'notebooks', 'output_file.csv', 'picolog_folder', 'README.md', 'requirements.txt']


## Read in Data

In [3]:
### list files in above directory
thisDir = os.getcwd()
os.listdir(thisDir+"/../")

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'notebooks',
 'output_file.csv',
 'picolog_folder',
 'README.md',
 'requirements.txt']

In [4]:
# Original credentials
bucket_remote = "GLADD"
org_remote = "PPE"
token_remote = "EO3dz3f5Bee1T45loRbpyk0SmHAkjOyB0qgKsv2Zc_NPW1_IBwy-odhRgVpPxBftVre63C4eS7V55EhCWbKqHQ=="
url_remote = "194.36.1.20:8086"

print("Connection details:")
print(f"bucket_remote: {bucket_remote}")
print(f"org_remote: {org_remote}")
print(f"token_remote: {token_remote}")
print(f"url_remote: {url_remote}")


Connection details:
bucket_remote: GLADD
org_remote: PPE
token_remote: EO3dz3f5Bee1T45loRbpyk0SmHAkjOyB0qgKsv2Zc_NPW1_IBwy-odhRgVpPxBftVre63C4eS7V55EhCWbKqHQ==
url_remote: 194.36.1.20:8086


## Format raw data
The following data takes in all of the files from the picologger folder,performs necessary calculations on them and puts them all in one csv file.

In [5]:
# Timestamp converter function
def TimeStampConverter(inStr, inPat):
    timeObj = None
    if inStr == "now":
        timeObj = datetime.now()
    else:
        timeObj = datetime.strptime(inStr, inPat)
    return timeObj.strftime("%Y-%m-%dT%H:%M:%SZ")

In [6]:
### derive values from picoLogger raw data
# Perform calculations function
def perform_calculations(input_folder, output_file, url_remote, token_remote, org_remote, timestamp_str):
    all_data = []

    timestamp = TimeStampConverter(timestamp_str, "%Y-%m-%d %H:%M:%S")

    for filename in os.listdir(input_folder):
        if filename.endswith('.csv'):
            file_path = os.path.join(input_folder, filename)
            data = pd.read_csv(file_path)

            
            # Calculate Vin Drop and GND Drop
            data['Vin Drop (V)'] = data['Vin+ Last (V)'] - data['Vin- Last (V)']
            data['GND Drop (V)'] = data['GND+ Last (V)'] - data['GND- Last (V)']
            
            # Calculate Resistance Vin(Ohms) and Total Resistance(mOhms)
            data['Resistance Vin(Ohms)'] = data['Vin Drop (V)'] / 10 / 5
            data['Resistance GND(Ohms)'] = data['GND Drop (V)'] / 10 / 5
            data['Total Resistance(mOhms)'] = (data['Resistance Vin(Ohms)'] + data['Resistance GND(Ohms)']) * 1000
            
            # Calculate Capacitor Equivalent leakage current(nA)
            data['Capacitor Equivalent leakage current(nA)'] = ((data['Capacitor leakage test Last (V)'] / 10) / (1 * 10 ** 6)) / 1000000
            
            # Calculate NTC value(Kohms)
            data['NTC value (Kohms)'] = 0.2 * 51 / data['NTC Last (V)']
                        
            # Prepare the new DataFrame with desired columns and values
            result_data = pd.DataFrame({
                'component': filename,
                'componentType': 'PCB',
                'stage': 'PCB_QC',
                'testType': 'HV_LV_TEST',
                'institution': 'Glasgow',
                'runNumber': data.shape[0],
                'date': date.today().strftime('%Y-%m-%d'),
                'passed': 'YES',
                'problems': 'False',
                'property1_value': 'B.masic',
                'property1_key': 'OPERATOR',
                'property2_value': 'INSTRUMENT',
                'property2_key': '',
                'property3_value': 'ANALYSIS_VERSION',
                'property3_key': '',
                'result1_key': 'VIN_DROP',
                'result1_value': data['Vin Drop (V)'],
                'result2_key': 'GND_DROP',
                'result2_value': data['GND Drop (V)'],
                'result3_key': 'EFFECTIVE RESISTANCE',
                'result3_value': data['Total Resistance(mOhms)'],
                'result4_key': 'HV_LEAKAGE',
                'result4_value': data['Capacitor leakage test Last (V)'],
                'result5_key': 'LEAKAGE_CURRENT (nA)',
                'result5_value': data['Capacitor Equivalent leakage current(nA)'],
                'result6_key': 'NTC_VOLTAGE',
                'result6_value': data['NTC Last (V)'],
                'result7_key': 'NTC_VALUE',
                'result7_value': data['NTC value (Kohms)']
            })
            
            all_data.append(result_data)
    
    if all_data:
        final_result = pd.concat(all_data, ignore_index=True)
        final_result.to_csv(output_file, index=False)
        print(f"Calculation completed. Output saved to {output_file}.")


In [7]:
### run conversion
input_folder = thisDir+"/../picolog_folder/"
output_file = thisDir+"/../output_file.csv"
timestamp_str = "2023-08-08 14:30:00"  # Replace this with your desired timestamp

perform_calculations(input_folder, output_file, url_remote, token_remote, org_remote, timestamp_str)



Calculation completed. Output saved to C:\Users\User\Documents\GitHub\picoLogger\notebooks/../output_file.csv.
